In [1]:
# Imports 
import os
import sys
sys.path.append("..")
os.environ["pyna_language"] = 'julia'
import pdb
import numpy as np
import openmdao.api as om
import pandas as pd
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset
%matplotlib inline
from pyNA.pyna import pyna

from IPython.display import clear_output
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# Inputs 
z_cb_lst = np.hstack((np.linspace(25, 250, 10), np.linspace(300, 750, 10)))

TS_cb_lst = dict()
TS_cb_lst['stca']= dict()
TS_cb_lst['stca']['V2'] = np.linspace(0.75, 1.0, 11)
TS_cb_lst['stca']['Vmax'] = np.linspace(0.4, 1.0, 25)
TS_cb_lst['a10'] = dict()
TS_cb_lst['a10']['V2'] = np.linspace(0.775, 1.0, 10)
TS_cb_lst['a10']['Vmax'] = np.linspace(0.675, 1.0, 14)

Vmax_lst = dict()
Vmax_lst['stca'] = dict()
Vmax_lst['stca']['V2'] = 197*0.51444444
Vmax_lst['stca']['Vmax'] = 250*0.51444444
Vmax_lst['a10'] = dict()
Vmax_lst['a10']['V2'] = 234.4*0.51444444
Vmax_lst['a10']['Vmax'] = 250*0.51444444

### Compute trajectories

In [3]:
# Compute trajectories for (z-TS) sweep 
if False:
        
    for ac_name in ['stca', 'a10']:

        for case in ['V2', 'Vmax']:
            
            py = pyna()
            
            py.case_name = ac_name
            py.ac_name = ac_name
            py.engine_deck_name = 'engine_deck_' + ac_name + '.csv'
            py.save_results = True
            py.output_directory_name = 'NOCB/' + case
            py.tolerance = 1e-6
            py.max_iter = 300
            py.TS_to = 1.0
            py.TS_vnrs = 1.0
            py.pkrot = True

            py.theta_flaps = 10.
            py.theta_slats = -6.
            
            TS = 1
            z_cb = 300.
            
            print("TS: ", TS)
            print("z_cb:", z_cb)

            py.TS_cb = TS
            py.z_cb = z_cb
            py.output_file_name = py.ac_name + '.sql'

            # Compute trajectory
            py.v_max = Vmax_lst[ac_name][case]

            converged = py.compute_trajectory(trajectory_mode='cutback', objective='t_end')

            # Save timeseries
            path_save_name = 'trajectory.csv'
            engine_save_name = 'engine.csv'
            py.save_time_series(py.path, py.airframe, path_save_name, engine_save_name)

            clear_output()

### Extract properties of the trajectories

In [4]:
# Compute noise time series
if False:
    
    k_rot_lst = dict()
    LD_climbout = dict()
    
    for ac_name in ['a10']:

        k_rot_lst[ac_name] = dict()
        LD_climbout[ac_name] = dict()
        
        for case in ['Vmax']:
            
            k_rot_lst[ac_name][case] = np.zeros((np.size(z_cb_lst), np.size(TS_cb_lst[ac_name][case])))
            LD_climbout[ac_name][case] = np.zeros((np.size(z_cb_lst), np.size(TS_cb_lst[ac_name][case])))
            
            py = pyna()
            
            py.output_directory_name = 'STCB/' + case
            py.case_name = ac_name
            py.ac_name = ac_name
            py.engine_deck_name = 'engine_deck_' + ac_name + '.csv'
            py.save_results = True
            py.output_directory_name = 'STCB/' + case
            
            for j,TS in enumerate(TS_cb_lst[ac_name][case]):
                for i, z_cb in enumerate(z_cb_lst):

                    py.output_file_name = py.ac_name + '_' + str(np.round(z_cb, 3)) + '_' + str(np.round(TS, 3)) + '.sql'
                    
                    print(ac_name, " ", case, " - TS: ", TS, "; z_cb: ", z_cb)

                    results = py.load_results(py.output_file_name)
                    
                    k_rot_lst[ac_name][case][i,j] = results.get_val('phases.groundroll.parameters:k_rot')
                    LD_climbout[ac_name][case][i,j] = np.max(results.get_val('phases.cutback.interpolated.c_l')/results.get_val('phases.cutback.interpolated.c_d'))

            np.save('../cases/'+ac_name+'/output/STCB/'+case+'/sweeps/k_rot.npy', k_rot_lst[ac_name][case])
            np.save('../cases/'+ac_name+'/output/STCB/'+case+'/sweeps/LD_climbout.npy', LD_climbout[ac_name][case])

            clear_output()

### Compute certification noise levels

In [5]:
# Compute noise time series
if True:
        
    for ac_name in ['stca']:

        for case in ['Vmax']:
#         for case in ['V2']:
            
            n_sl = 131
            py = pyna()
            py.ac_name = ac_name
            py.case_name = ac_name
            py.output_directory_name = 'NOCB/' + case
            
            py.output_file_name = py.ac_name + '.sql'
            py.engine_timeseries_name = 'engine.csv'
            py.trajectory_file_name = 'trajectory.csv'

            py.observer_lst = ['contours',]

            x_observer_lst = np.zeros((n_sl+1, 3))
            x_observer_lst[:-1, 0] = np.linspace(0, 6500, n_sl)
            x_observer_lst[:-1, 1] = 450*np.ones(n_sl)
            x_observer_lst[:, 2] = 4*0.3048*np.ones(n_sl+1)
            x_observer_lst[-1,0] = 6500.

            py.x_observer_array = x_observer_lst

            py.compute_noise_timeseries()

            np.save('../cases/'+ac_name+'/output/NOCB/'+case+'/epnl.npy', py.noise_timeseries.get_val('noise.epnl'))

            clear_output()

/Users/laurensvoet/Library/Python/3.7/lib/python/site-packages/openmdao/recorders/sqlite_recorder.py:224: UserWarning:The existing case recorder file, /Users/laurensvoet/Documents/Research/pyNA/pyNA/cases/stca/dymos_solution.db, is being overwritten.


Model viewer data has already has already been recorded for Driver.


RuntimeError: 'noise' <class JuliaExplicitComp>: Error calling compute(), <PyCall.jlwrap (in a Julia function called from Python)
JULIA: UndefVarError: kkk not defined
Stacktrace:
 [1] (::var"#noise_model#17")(::PynaInterpolations, ::Dict{Any,Any}, ::PyObject, ::PyObject, ::Dict{Any,Any}, ::String, ::Array{Float64,1}) at /Users/laurensvoet/Documents/Research/pyNA/pyNA/src/noise_src_jl/noise_model.jl:263
 [2] compute!(::NoiseModel{PynaInterpolations,Dict{Any,Any},PyObject,PyObject,Int64,Dict{Any,Any},String,var"#noise_model#17",var"#16#18"{Dict{Any,Any},PyObject,PyObject,Dict{Any,Any},String,var"#noise_model#17",PynaInterpolations}}, ::PyDict{String,PyArray,true}, ::PyDict{String,PyArray,true}) at /Users/laurensvoet/Documents/Research/pyNA/pyNA/src/noise_src_jl/noise_model.jl:0
 [3] compute!(::BigInt, ::PyDict{String,PyArray,true}, ::PyDict{String,PyArray,true}) at /Users/laurensvoet/Documents/Research/OpenMDAO.jl/src/OpenMDAO.jl:84
 [4] (::PyCall.FuncWrapper{Tuple{Integer,PyDict{String